# Twitter Sentiment Analysis for Indian Election 2019

**Abstract**<br>
The goal of this project is to do sentiment analysis for the Indian Elections. The data used is the tweets that are extracted from Twitter. The BJP and Congress are the two major political parties that will be contesting the election. The dataset will consist of tweets for both the parties. The tweets will be labeled as positive or negative based on the sentiment score obtained using Textblob library. This data will be used to build models that can classify new tweets as positive or negative. The models built are a Bidirectional RNN and GloVe word embedding model.

**Implementation**<br>

In [6]:
import os
import pandas as pd
import tweepy
import re
import string
from textblob import TextBlob
import preprocessor as p
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
import pandas as pd
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, SimpleRNN,Input
from keras.models import Sequential,Model
from keras.preprocessing import sequence
from keras.layers import Dense,Dropout
from keras.layers import Embedding, Flatten, Dense,Conv1D,MaxPooling1D
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import itertools    
import seaborn as sns
from sklearn.metrics import confusion_matrix
from keras.utils import to_categorical
from collections import Counter
import tensorflow as tf
from keras.layers import LSTM, Bidirectional, Dropout

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Abhishek\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**Data Creation**

We use Tweepy API to access Twitter and download tweets. Tweepy supports accessing Twitter via Basic Authentication and the newer method, OAuth. Twitter has stopped accepting Basic Authentication so OAuth is now the only way to use the Twitter API.
The below code downloads the tweets from Twitter based on the keyword that we pass. The tweets sentiment score is obtained using the textblog library. The Tweets are then preprocessed. The preprocessing involved removing emoticons, removing stopwords.

In [ ]:
consumer_key= '9oO3eQOBkuvCRPqMsFvnShRrq'
consumer_secret= 'BMWGbdC05jDcsWU5oI7AouWvwWmi46b2bD8zlnWXaaRC7832ep'

access_token='313324341-yQa0jL5IWmUKT15M6qM53uGeGW7FGcy1xAgx5Usy'
access_token_secret='OyjmhcMCbxGqBQAWzq12S0zrGYUvjChsZKavMYmPCAlrE'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

#file location changed to "data/telemedicine_data_extraction/" for clearer path
congress_tweets = "C:/Users/Abhishek/Election Twitter Sentiment analysis/congress_test.csv"
bjp_tweets = "C:/Users/Abhishek/Election Twitter Sentiment analysis/bjp_test_new.csv"

#set two date variables for date range
start_date = '2019-04-1'
end_date = '2019-04-20'

**Data cleaning scripts**

In [ ]:
# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
 
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
#Emoji patterns
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
 
#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

#mrhod clean_tweets()
def clean_tweets(tweet):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)
 
    #after tweepy preprocessing the colon left remain after removing mentions
    #or RT sign in the beginning of the tweet
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
 
 
    #remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
 
    #filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []
 
    #looping through conditions
    for w in word_tokens:
        #check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)
    #print(word_tokens)
    #print(filtered_sentence)

In [ ]:
#method write_tweets()
def write_tweets(keyword, file):
    # If the file exists, then read the existing data from the CSV file.
    if os.path.exists(file):
        df = pd.read_csv(file, header=0)
    else:
        df = pd.DataFrame(columns=COLS)
    #page attribute in tweepy.cursor and iteration
    for page in tweepy.Cursor(api.search, q=keyword,
                              count=200, include_rts=False, since=start_date).pages(50):
        for status in page:
            new_entry = []
            status = status._json
 
            ## check whether the tweet is in english or skip to the next tweet
            if status['lang'] != 'en':
                continue
 
            #when run the code, below code replaces the retweet amount and
            #no of favorires that are changed since last download.
            if status['created_at'] in df['created_at'].values:
                i = df.loc[df['created_at'] == status['created_at']].index[0]
                if status['favorite_count'] != df.at[i, 'favorite_count'] or \
                   status['retweet_count'] != df.at[i, 'retweet_count']:
                    df.at[i, 'favorite_count'] = status['favorite_count']
                    df.at[i, 'retweet_count'] = status['retweet_count']
                continue
 
 
           #tweepy preprocessing called for basic preprocessing
            #clean_text = p.clean(status['text'])
 
            #call clean_tweet method for extra preprocessing
            filtered_tweet=clean_tweets(status['text'])
 
            #pass textBlob method for sentiment calculations
            blob = TextBlob(filtered_tweet)
            Sentiment = blob.sentiment
 
            #seperate polarity and subjectivity in to two variables
            polarity = Sentiment.polarity
            subjectivity = Sentiment.subjectivity
 
            #new entry append
            new_entry += [status['id'], status['created_at'],
                          status['source'], status['text'],filtered_tweet, Sentiment,polarity,subjectivity, status['lang'],
                          status['favorite_count'], status['retweet_count']]
 
            #to append original author of the tweet
            new_entry.append(status['user']['screen_name'])
 
            try:
                is_sensitive = status['possibly_sensitive']
            except KeyError:
                is_sensitive = None
            new_entry.append(is_sensitive)
 
            # hashtagas and mentiones are saved using comma separted
            hashtags = ", ".join([hashtag_item['text'] for hashtag_item in status['entities']['hashtags']])
            new_entry.append(hashtags)
            mentions = ", ".join([mention['screen_name'] for mention in status['entities']['user_mentions']])
            new_entry.append(mentions)
 
            #get location of the tweet if possible
            try:
                location = status['user']['location']
            except TypeError:
                location = ''
            new_entry.append(location)
 
            try:
                coordinates = [coord for loc in status['place']['bounding_box']['coordinates'] for coord in loc]
            except TypeError:
                coordinates = None
            new_entry.append(coordinates)
 
            single_tweet_df = pd.DataFrame([new_entry], columns=COLS)
            df = df.append(single_tweet_df, ignore_index=True)
            csvFile = open(file, 'a' ,encoding='utf-8')
    df.to_csv(csvFile, mode='a', columns=COLS, index=False, encoding="utf-8")

In [ ]:
#declare keywords as a query for three categories
Congress_keywords = '#IndianNationalCongress OR #RahulGandhi OR #SoniaGandhi OR #INC' 
BJP_keywords = '#BJP OR #Modi OR #AmitShah OR #BhartiyaJantaParty'

Creates two CSV files. First saves tweets for BJP and second saves tweets for Congress.

In [ ]:
#call main method passing keywords and file path
write_tweets(Congress_keywords,  congress_tweets)
write_tweets(BJP_keywords, bjp_tweets)

**LABELING TWEETS AS POSITIVE NEGATIVE**<br>


The tweepy libary gives out sentiment polarity in the range of -1 to +1. For our topic of election prediction the neutral tweets would be of no use as they will not provide any valuable information. Thus for simplicity purpose I have labeled tweets as only positive and negative. Tweets with polarity less than 0 will be labelled negative(0) and greater than 0 will be positive(1)

In [ ]:
bjp_df['polarity'] = bjp_df['polarity'].apply(lambda x: 1 if x > 0 else 0)
congress_df['polarity'] = congress_df['polarity'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
bjp_df['polarity'].value_counts()

![alt text](1.png "Title")

In [ ]:
congress_df['polarity'].value_counts()

![alt text](2.png "Title")

## **RESAMPLING THE DATA** <br>
Since the ratio of the negative tweets to positive tweets is not proportional. Our data set is not balanced. This will create a bias while training the model. To avoid this I have resampled the data. New data was downloaded from twitter using the above procedure. For both the parties only positive tweets were sampled and appened to the main files to balance the data. After balancing the data. The count of positive and negative tweets for both the parties is as follows. The code for the resampling procedure can be found in the notebook Data_Labeling.ipynb

![alt text](4.png "Title")

**CREATING FINAL DATASET**

In [ ]:
frames = [bjp, congress]
election_data = pd.concat(frames)

The final dataset that will be used for our analysis saved in a csv file. That file can be loaded used to run our models. The final dataset looks as follows.

![alt text](5.png "Title")

**TOKENIZING DATA**

We tokenize the text and keep the maximum length of the the vector 1000.

![alt text](6.png "Title")

**TRAIN TEST SPLIT WITH 80:20 RATIO**

In [ ]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(.20 * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

**CREATING EMBEDDING MATRIX WITH HELP OF PRETRAINED MODEL: GLOVE**

Word Embeddings are text converted into numbers. There are number of ways to represent the numeric forms.<br>

Types of embeddings: Frequency based, Prediction based.<br>Frequency Based: Tf-idf, Co-occurrence matrix<br>

Prediction-Based: BOW, Skip-gram model
Using Pre-trained word vectors: Word2vec, Glove

Word Embedding is done for the experiment with the pre trained word vector Glove.

Glove version used : 100-dimensional GloVe embeddings of 400k words computed on a 2014 dump of English Wikipedia. Training is performed on an aggregated global word-word co-occurrence matrix, giving us a vector space with meaningful substructures

![alt text](7.png "Title")

In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Creating an embedding layer using GloVe

In [ ]:
embedding_layer = Embedding(len(word_index) + 1,
                            100,
                            weights=[embedding_matrix],
                            input_length=1000,
                            trainable=False)

# Model 1

**Glove Word Embedding model**

GloVe is an unsupervised learning algorithm for obtaining vector representations for words. Training is performed on aggregated global word-word co-occurrence statistics from a corpus, and the resulting representations showcase inter-esting linear substructures of the word vector space. GloVe can be used to find relations between words like synonyms, company - product relations, zip codes, and cities etc. It is also used by the spaCy model to build semantic word em-beddings/feature vectors while computing the top list words that match with distance measures such as Cosine Similar-ity and Euclidean distance approach.

In [22]:
def model_creation():
  input_layer = Input(shape=(1000,), dtype='int32')
  embed_layer = embedding_layer(input_layer)
  x = Dense(100,activation='relu')(embed_layer)
  x = Dense(50,activation='relu', kernel_regularizer=keras.regularizers.l2(0.002))(x)
  x = Flatten()(x)
  x = Dense(50,activation='relu', kernel_regularizer=keras.regularizers.l2(0.002))(x)
  x = Dropout(0.5)(x)
  x = Dense(50, activation='relu')(x)
  x = Dropout(0.5)(x)
  #x = Dense(512, activation='relu')(x)
  #x = Dropout(0.4)(x)
  final_layer = Dense(1, activation='sigmoid')(x)
  opt = keras.optimizers.Adam(lr= learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
  model = Model(input_layer,final_layer)
  model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['acc'])
  return model

**MODEL 1 Architecture**

In [ ]:
learning_rate = 0.0001
batch_size = 1024
epochs = 10
model_glove = model_creation()

![alt text](8.png "Title")

![alt text](9.png "Title")

**SAVE BEST MODEL AND WEIGHTS for Model1**

In [ ]:
# serialize model to JSON
model_json = model_glove.to_json()
with open(".\\SavedModels\\Model_glove.h5", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_glove.save_weights(".\\SavedModels\\Weights_glove.h5")

**MODEL1 LOSS AND ACCURAY**

![alt text](10.png "Title")

**MODEL1 PERFORMANCE**

In [ ]:
def plot_modelacc(fit_model):
    with plt.style.context('ggplot'):
            plt.plot(fit_model.history['acc'])
            plt.plot(fit_model.history['val_acc'])
            plt.ylim(0,1)
            plt.title("MODEL ACCURACY")
            plt.xlabel("# of EPOCHS")
            plt.ylabel("ACCURACY")
            plt.legend(['train', 'test'], loc='upper left')
    return plt.show()

def plot_model_loss(fit_model):
    with plt.style.context('ggplot'):
            plt.plot(fit_model.history['loss'])
            plt.plot(fit_model.history['val_loss'])
            plt.title("MODEL LOSS")
            plt.xlabel("# of EPOCHS")
            plt.ylabel("LOSS")
            plt.legend(['train', 'test'], loc='upper left')
    return plt.show()

![alt text](11.png "Title")

**CONFUSION MATRIX**<br>
A confusion matrix will show us the how the model predicted with respect to the acutal output. 

![alt text](12.png "Title")

True Positives: 870 (Predicted True and True in reality)<br>
True Negative: 1141(Predicted False and False in realtity)<br>
False Positive: 33 (Predicted Positve but Negative in reality)<br>
False Negative: 29 (Predicted Negative but Positive in reality)

# Model 2

**Bidirectional RNN model**

Bidirectional Recurrent Neural Networks (BRNN) connect two hidden layers of opposite directions to the same output. With this form of generative deep learning, the output layer can get information from past (backwards) and future (forward) states simultaneously.Invented in 1997 by Schuster and Paliwal,BRNNs were introduced to increase the amount of input information available to the network. For example, multilayer perceptron (MLPs) and time delay neural network (TDNNs) have limitations on the input data flexibility, as they require their input data to be fixed. Standard recurrent neural network (RNNs) also have restrictions as the future input information cannot be reached from the current state. On the contrary, BRNNs do not require their input data to be fixed. Moreover, their future input information is reachable from the current state.

BRNN are especially useful when the context of the input is needed. For example, in handwriting recognition, the performance can be enhanced by knowledge of the letters located before and after the current letter.

**MODEL 1  Architecture**

![alt text](13.png "Title")
![alt text](14.png "Title")

**SAVING BEST MODEL2 AND ITS WEIGHTS**

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open(".\\SavedModels\\Model_Bidir_LSTM.h5", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(".\\SavedModels\\Weights_bidir_LSTM.h5")
print("Saved model to disk")

**MODEL 2 LOSS AND ACCURACY**

![alt text](15.png "Title")

![alt text](16.png "Title")

**MODEL 2 CONFUSION MATRIX**

![alt text](17.png "Title")

True Positives: 887(Predicted True and True in reality)
True Negative: 1140(Predicted False and False in realtity)
False Positive: 35 (Predicted Positve but Negative in reality)
False Negative: 11 (Predicted Negative but Positive in reality)

**PREDICTION USING THE BEST MODEL**

The models were compared based on the Test loss and Test Accuracy. The Bidirectional RNN performed slightly better than the GloVe model. The RNN despite its simple architec-ture performed better than the Glove model. We use the Bidirectional RNN to make the predictions for the tweets that will be used to infer election results.

Load the test data on which the predictions will be made using our best model. The data for both the parties was collected using the same procedure like above.

In [ ]:
congress_test = pd.read_csv('congress_test.csv')
bjp_test = pd.read_csv('bjp_test.csv')

We took equal samples for both the files. We took 2000 tweets for Congress and 2000 for BJP. The party that gets the most number of positive votes can be infered to have the higest probablity of winning the 2019 English.

In [ ]:
congress_test =congress_test[:2000]
bjp_test = bjp_test[0:2000]

Tokenize the tweets in the same was that were used for the Bidirectional RNN model.

In [ ]:
congress_inputs = tokenze_data(congress_inputs)
bjp_inputs = tokenze_data(bjp_inputs)

**LOAD THE BEST MODEL (BIDIRECTIONAL LSTM)**

In [ ]:
from keras.models import model_from_json
# load json and create model
json_file = open(".\\SavedModels\\Model_Bidir_LSTM.h5", 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(".\\SavedModels\\Weights_bidir_LSTM.h5")
print("Loaded model from disk")

**SENTIMENT PREDICTION USING THE MODEL**

In [ ]:
congress_prediction = loaded_model.predict(congress_inputs)
bjp_prediction = loaded_model.predict(bjp_inputs)

If the probabilty of the outcome is greater than 0.5 for any class then the sentiment belongs to that particular class. Since we are concerned with only the count of positive sentiments. We will check the second column variables for our inference. 

In [ ]:
congress_pred = (congress_prediction>0.5)
bjp_pred = (bjp_prediction>0.5)

def get_predictions(party_pred):
    x = 0
    for i in party_pred:
        if(i[1]==True):
            x+=1
    return x

![alt text](18.png "Title")

**CONCLUSION**

Just like the training data the majority of the tweets have a negative sentiment attached to them. After feeding 2000 tweets for both the Congress and BJP. The model predicted that BJP has 660 positive tweets while Congress has 416 positive tweets.<br><br> This indicated that the contest this year would be close and the chances of BJP winning on Majority like the 2015 elections are less. This has been corraborated by the poor perfomace of the BJP in the recent state elections where the lost power in three Major Hindi speaking states Rajasthan, Madhya Pradesh and Chattishgarh. <br><br> 

**FUTURE SCOPE**

For this project only, a small sample of twitter data was considered for the analysis. It is difficult to give an estimate based on the limited amount of information we had access to. For future work, we can start by increasing the size of our dataset. In addition to Twitter, data can also be obtained from websites like Facebook, News websites. Apart from these we can try different models like Bidirectional RNN with attention mechanism. We can implement BERT which is currently the state of the art for solving various Natural Language Pro-cessing problems.

**LISCENCE**

**REFERENCES**

[1] Sepp Hochreiter and Jurgen Schmidhuber, “Long short- ¨ term memory,” Neural computation, vol. 9, no. 8, pp. 1735–1780,1997.<br>
[2] Mike Schuster and Kuldip K Paliwal, “Bidirectional recurrentneural networks,” Signal Processing, IEEE Transactions on, vol.     45, no. 11, pp. 2673–2681, 1997.<br>
[3] Jeffrey Pennington, Richard Socher, Christopher D. Manning.GloVe: Global Vectors for Word Representation <br>
[4] Apoorv Agarwal Boyi Xie Ilia Vovsha Owen Rambow Rebecca Passonneau Sentiment Analysis of Twitter Data <br>
[5] Alex Graves and Jurgen Schmidhuber, “Framewise ¨ phoneme classification with bidirectional LSTM and other neural network
    architectures,” Neural Networks, vol. 18, no. 5, pp. 602–610,2005